<img src="../qbies.png" style="width: 100px;padding-right:5px;padding-top:1px;padding-left:5px;" align="left"/>

<p style="padding-left:125px;padding-top: 30px";><font size="+2"><b> Practical Guidance - Functions
    </b></font><p>

# Coding Guidelines 
There are a few coding guidelines governing the structure we recommend in a lambda function and these are laid out in this section.

## Function Names, Variables & Arguments
It is good practice to use descriptive names for functions/variables/arguments to make our code clearer for the reader. 



Use the **lowerCamel** naming convention for function names, arguments & variables. This means that a variable with one word in it (e.g. price) is written price, whereas a word with two or more words is written by capitalizing the second and subsequent words e.g. 
* size is written **size** 
* last price is written **lastPrice** 
* last fill size is written **lastFillSize**. 


This makes it easier to read than lastfillsize.

You also have license to remove some letters in a variable, provided the word is still easily understood and there is no ambiguity. 
Some examples:

| Meaning     |    Bad Examples  |    Good Examples           |
|:---|:---|:---|
|"price"     |    p         | price               
|"price"           |    Price        | px  (common abbreviation in Finance)      
|"size"       |   s          | size               
|"size"      |   Size        |sz  (common abbreviation in Finance) 
|"last price"  |  lp   | lastPrice   
| "last price" |  LastPrice   |  lastpx 
|"get last size" |glz            | getLastSize 
| "get last size"|            | getLastSz 
|"ensure string"| ensurestring| ensureString 
|"ensure string"| es| ensureStr (str is usually understood as string)

## Layout

A function should be written in a clear, logical fashion so that it is easy for somebody else to read. Well-named functions, variables and arguments are the first part of this. 

Here are some other general conventions that should be followed when writing functions:

* **Explicit Argument Declaration**: x,y and z are great, as we all know, and if we have a very short function in which it is blatantly obvious what x,y and z are then it is ok to not declare them. If, on the other hand there is any room for ambiguity then the argument(s) should probably be declared.

* **Don't do too much on one line**: If you find that a line has 2, 3 or 4 levels of bracket nesting, breaking it up into intermediary steps can often make it easier to read.

* **Use indenting for "if"s**: This makes it easier to tell where the if starts and ends

* **Comments**: Use comments to provide extra information to the reader that might not be clear from reading the code

* **Avoid Hard Coding**: Always use global variables for configuration as opposed to hard coding them into the function. This allows the developer to easily see the variables at the top of the file, and allows you to alter the variables while the process is running without having to redefine the function.

* **Don't use `:` to return at the end**: This is a matter of style preference, and for consistency we all follow this rule. You should only use `:` to return EARLY from a function. You don't need one in the last expression unless it is at the end of an if

Let's take the following function as an example:

In [1]:
getPrevailingQuote:{
   if[not x in `JPM`GE`MSFT;'"Invalid sym ",string x];
   aj[`sym`time;select from Quote where sym=x;update`g#sym from select from Trade where sym=x,time within 09:30 16:00]}  

It has the following flaws:

* *Not immediately clear how many arguments the function takes, and what they represent* - namely using implicit variables for a big function should be avoided, instead we should use explicit definition. If in doubt, be explicit.

* *Hard codes filters - we've hard-coded the time range we're looking at to be 09:30-16:00*. Instead, we should either pass these as parameters or use global variables.

* *Difficult to read - the code is all squashed together on the same line*. Ideally we should indent for new parameters where appropriate, or define in local scope and use subsequently for long expressions.

* *Not much room for comments, and also no comments included!* Where assumptions are made, or code is less clear we should comment.

* *Difficult to debug due to lack of intermediary steps.* By splitting the function into smaller more discrete steps we can debug more easily.

Now if we take the following example:

In [2]:
.cfg.VALID_SYMS:`JPM`GE`MSFT;
.cfg.MARKET_HOURS:09:30 16:00;

getPrevailingQuote:{[s] 
 if[not s in .cfg.VALID_SYMS;
   '"Invalid sym ",string s];
 quotes:select from Quote where sym=s;
 trades:select from Trade where sym=s,time within .cfg.MARKET_HOURS;  // exclude trades outside of market hours
 aj[`sym`time;trades;update`g#sym from quotes]                        // `g# improves performance
 }

We can see that it is easier to read for somebody who hasn't seen it before, and will be easier to debug and maintain.

<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>In a q script, the closing bracket of a function/lambda <b>must be</b> at least one space away from the margin, otherwise the script will error. This is one of the rare cases where kdb+/q requires whitespace! </i></p>

In [2]:
/ – Fails to load! 
getTrades:{[s]
 select from Trade where sym=s
}

[0;31mparse error[0m: [0;31m{[0m

In [3]:
/ – Works! 
getTrades:{[s]
 select from Trade where sym=s
 }       // Closing bracket must be off the margin

# Advice for writing functions

## Getting started 

##### Define the Requirements

The best place to start is to fully define the requirements for the function: 
* What input is this expected to take? - consider behaviour with both atoms and lists
* Is this input likely to change/grow? (see **Function Parameters >8** for more discussion on this)
* Are there any "default" parameters? (see **Function Parameters >8** for more discussion on this)
* What edge cases are there to that input? (if any) 
* What behaviour is expected/required on unexpected input?
* Where will this function run? 
    * This helps determine if logging is required e.g. in a function for data loading we should include logging to help debug, however, if it is running in a latency sensitive environment we probably won't include logging.
    * This also helps to determine if there are any performance requirements for the function 
* What output is required/expected?


##### Start with pseudocode 
Once you have the requirements defined, it then becomes a case of sketching out the pseudocode for the problem. Consider the following: 
* What data do I need for each step? 
* What is the most efficient way of performing the operations to consistently reduce the data/timing load etc? 
* Do I have all cases accounted for? 
* Is my output consistent?

##### Define the code tests (Test driven development) 

Test all known requirements 

##### Write the code!

##### TEST 
Many institutions have their own internal testing frameworks, methodologies etc. If you're not working within an already existing framework and aren't keen to write your own, Kx provides a free unit-testing library called [qcumber](https://code.kx.com/developer/qcumber/), that offers assertion based testing, benchmarking capabilities, and property based testing. This is available for download and use [here](https://code.kx.com/developer/libraries-scripts/).

Iterate between all of these until complete 

## Worked Example 

        Request:*Write a function that returns a null type of whatever is passed*

Requirements:
* Expected input - not specified so must be able to cope with both atoms and lists 
* Edge Cases: 
    * Null types don't exist for Booleans and Bytes - what is expected behaviour here? 
    * What is expected behaviour with General Lists? 
    * All other types e.g. functions tables etc. 

Responses to initial queries: 
* General lists - this should throw an error '"No stinking General Lists!" 
* Booleans,Bytes - write to stdErr "This might be a bad idea... " and return the equivalent 0 value 
* Other types - return an empty list. 

Pseudocoding: 
* Put all our exceptions first to avoid code errors: 
    1. Check the type of the input - account for both lists and atoms
    * Check for input types > 19 (see [datatypes](https://code.kx.com/q/basics/datatypes/)) since returning an empty list is very lightweight, so for efficiency we should put that first. 
    * Check for input of a general lists - throw error  if so 
    * Check Boolean/Byte type - if so stderr warning and return the 0 value 
    * All else, return the null: 
        * Use the type of the input to cast the null long `0N`
        * (Alternatively, could make this a list and index beyond the list to get a null of the same type - this would be handier if we *did* want to handle general lists by returning the type of the first input for example) 
 

Test cases: 
* Test with explicit edge cases Boolean, General Lists, types > 19
* Test with other inputs e.g. Characters, symbols, times etc
* Test with atoms and simple lists

In [4]:
//Write the code! 
returnNull:{[input]  tp:abs type input;   //1. getting the abs type to account for lists 
                     if[tp > 19;          //2. checking for types > 19
                         :()];            //early force return the empty list for non data based types
                     if[0=tp;             //3. check if General list
                         '"No stinking General Lists!"];
                     if[tp in 1 4h;       //4. check if Boolean/Guid
                         -2 "This might be a bad idea ...";
                         :tp$0];          //return the 0 value equiv for booleans and guids
                     tp$0N
     } 

In [5]:
//Test the code!
returnNull 0b

This might be a bad idea ...


0b


In [5]:
returnNull (0;`a)

[0;31mNo stinking General Lists![0m: [0;31mNo stinking General Lists![0m

In [6]:
returnNull {x+y} //returns empty list

In [7]:
returnNull 4e        //test atom 
returnNull 41 2 1e   //test list

0Ne


0Ne


In [8]:
returnNull "this"  //string 

"\000"


In [8]:
returnNull 1?0Ng   //guid

[0;31mtype[0m: [0;31mtype[0m

Looks like we caught two edge cases in testing! 
* Character input doesn't look quite right but didn't throw an error 
* GUIDs threw an error

We now need to fix the output for these two types - there's nothing here that seems like we need to go back and ask about. 

In [9]:
//Write the code! 
returnNull:{[input]  tp:abs type input;   
                     if[tp > 19;          
                         :()];            
                     if[0=tp;             
                         '"No stinking General Lists!"];
                     if[tp in 1 4h;       
                         -2 "This might be a bad idea ...";
                         :tp$0];
                     $[tp = 10;
                            " "; 
                         tp = 2; 
                            0Ng; 
                             tp$0N]                   
     } 

In [10]:
//retest!
returnNull 0b
returnNull {x+y} 
returnNull 4e        
returnNull 41 2 1e 
returnNull 1?0Ng
returnNull "this"
returnNull 09:00:00

This might be a bad idea ...


0b


0Ne


0Ne


00000000-0000-0000-0000-000000000000


" "


0Nv


Great looks like we're code complete!

For reference, a more general way to do this (if we didn't have specific error and standard error write requirements for particular input) would be the below: 

In [11]:
returnNull:{[input] ((),input)[0W]}  //making the input a list if not already and indexing beyond the domain

In [12]:
returnNull 0b
returnNull {x+y} 
returnNull 4e        
returnNull 41 2 1e 
returnNull 1?0Ng
returnNull "this"
returnNull 09:00:00

0b


0Ne


0Ne


00000000-0000-0000-0000-000000000000


" "


0Nv


## Avoid scope clashes

Avoid updating Global scope variables unless this is something you have to do (e.g. inserting data into a table). This means we should avoid the following syntaxes: 
* Global assignment using `::`
* Using the keyword `set` 
* Amend in place/Amend by reference syntax - usually familiar to C developers e.g. `x+:1` - this updates a *global* `x`, not the `x` in the function scope! 

In [13]:
f:{GLOBAL_X::x} //avoid double colon assignment unless you know you want to do this! 

In [14]:
f:{`SETTING_GLOBAL set x} //again, setting a global

In [15]:
m:100
myCount:{m:0;{m+:1} each x;m}   //this is insidious, just avoid this syntax until you're more familiar 
myCount[til 10]
m

0


110


In [16]:
m:100
myCount:{m:0;{[m]m:m+1} each x;m} //explicit parameter declaration and assignment
myCount[til 10]
m

0


100


# Function Parameters >8 
While kdb+/q limits the number of inputs to a function to 8, this isn't the restriction it first appears to be since we can pass arguments like lists or dictionaries. 

## Lists 
If we wanted to pass more arguments than 8 we can do so as a list: 

In [17]:
f:{[listArg] var2:listArg[1]; 
            var9:listArg[8];
            -1"Variables:",string[var2]," ",string[var9]; 
            var9+2*var2
    }
f[til 10]

Variables:1 8


10


## Dictionaries
Better yet, we can pass a dictionary - since this stores key value pairs, we can retrieve variables by their reference name making code more readable and supportable. 

In [18]:
show d:`v1`v2`v3`v4`v5`v6`v7`v8`v9`v10!til 10;
f:{[varDict] -1"Variables:",string[varDict[`v2]]," ",string[varDict[`v9]]; 
            varDict[`v9]+2*varDict[`v2]
    }
f[d]

v1 | 0
v2 | 1
v3 | 2
v4 | 3
v5 | 4
v6 | 5
v7 | 6
v8 | 7
v9 | 8
v10| 9
Variables:1 8


10


The beauty of using dictionaries is that we can modify our dictionary to only have the two keys needed for our function and the code would still run. 

In [19]:
show d2:`v2`v9! 10 20
f[d2]

v2| 10
v9| 20
Variables:10 20


40


Dictionaries also allow us to set default values for input if it's not passed: 

In [20]:
f:{[varDict] -1"Variables:",string[varDict[`v2]]," ",string[varDict[`v9]]; 
            default: `v9`v2!100 10;
            varDict: default,varDict;
            varDict[`v9]+2*varDict[`v2]
    }
f[enlist[`]!enlist[::]]  //no assigned values for v9 or v2 - empty dictionary - uses default
f[d2]                    //values passed and used
f[enlist[`v9]!enlist 10] //only one passed


Variables::: ::
Variables:10 20
Variables: 10


120


40


30


<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:30px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Use dictionaries for any functions where the number of inputs will or could change! If you have a function with three inputs in your code base, and you then update the function to take only two inputs, every instance in your code base will now throw rank errors! However, if you decide you want to change the way a function written with a dictionary input works, you don't necessarily need to change every line in the code base to modify the way the function is called! </i></p>